# 作業4：使用中文維基百科，利用pyserini建立反向索引，並取得 "蝙蝠俠" 一詞的IDF值

In [ ]:
from google.colab import drive
import pandas as pd
import json
drive.mount('/content/gdrive') # 此處需要登入google帳號

Mounted at /content/gdrive


## Load dataset

In [ ]:
data = pd.read_json("/content/gdrive/MyDrive/wiki_2021_08_05_1215639.json")

In [ ]:
data.head()

,id,title,articles
0,1,福斯家庭電影頻道,福斯家庭電影頻道（）是一個播放電影的亞洲電視頻道，由福斯國際頻道持有與出品，STAR衛視發佈...
1,2,六羰基鎢,六羰基鎢是一種配位化合物，化學式爲W(CO)。經過它得到了第一個雙氫配合物。它是無色固體，和...
2,3,吉米·史奴卡,吉米·史奴卡（，），本名詹姆斯·威廉·瑞荷（），是世界摔角娛樂（WWE）旗下退休職業摔角選手...
3,4,達文波特-欣策爾序列,在組合數學中，達文波特–欣策爾序列是指對任意兩個符號交替出現的次數作出限制的序列。達文波特–...
4,5,米基爾·安祖·莫亞,米基爾‧安祖‧莫亞（，)簡稱 莫亞 。西班牙足球運動員，司職門將。莫亞生於西班牙馬約卡島上的...


## 計算所有文檔的數量

In [ ]:
# 所有文檔的數量
total_docs = data.shape[0]
print(total_docs)

1215639


## 資料前處理

In [ ]:
dict_ = {}
json_list = []
for idx, row in data.iterrows():

  dict_['id'] = row['id']
  dict_['contents'] = row['articles']
  json_list.append(dict_)
  dict_ = {}

In [ ]:
# 將Python 的物件資料轉換成為JSON 物件
with open('/content/gdrive/MyDrive/IR_HW4/input/json_data.json', 'w') as f:
    json.dump(json_list, f, indent = 4) 

In [ ]:
pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.8 MB/s eta 0:00:00


## 建立反向索引

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input '/content/gdrive/MyDrive/IR_HW4/input' \
  --language zh \
  --index '/content/gdrive/MyDrive/IR_HW4/output' \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-03-15 01:47:38,185 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-03-15 01:47:38,188 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-03-15 01:47:38,188 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-03-15 01:47:38,189 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: /content/gdrive/MyDrive/IR_HW4/input
2023-03-15 01:47:38,189 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2023-03-15 01:47:38,190 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-03-15 01:47:38,190 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 1
2023-03-15 01:47:38,191 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: zh
2023-03-15 01:47:38,191 INFO  [main] index.In

## 進行反向索引搜尋

In [ ]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('/content/gdrive/MyDrive/IR_HW4/output')
searcher.set_language('zh')
hits = searcher.search('蝙蝠俠')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    print(hits[i].raw)

 1 239924 13.91470
{
  "id" : 239924,
  "contents" : "蝙蝠俠的腰帶裝備是DC漫畫的虛構超級英雄角色蝙蝠俠着裝的一個特色。除了蝙蝠俠之外，各個版本的羅賓、蝙蝠女孩以及蝙蝠家族的其他成員都有類似的腰帶。蝙蝠俠歷史研究學者認爲是加德納·福克斯，除比爾·芬格以外的“蝙蝠俠系列”的第一作者，首次在《偵探漫畫》第29期（1939年7月出版）裏介紹了腰帶裝備的概念。 在腰帶裝備的首次亮相中，第一個被介紹的裝備是“含有窒息氣體的氣囊”。 在腰帶裝備亮相後的兩個故事之後，加德納·福克斯在《偵探漫畫》第31期（1939年9月出版）的“蝙蝠俠對戰吸血鬼”中撰寫了第一個以“蝙蝠”爲主題的武器裝備——。直到1989年，《蝙蝠俠》的絕大部分創作者都將蝙蝠俠的腰帶裝備繪製成一條簡單的黃色腰帶，上面有帶扣和一些膠囊或圓筒。1986年，弗蘭克·米勒在蝙蝠俠一個《蝙蝠俠：黑暗騎士歸來》中爲蝙蝠俠的腰帶添加了軍裝風格。在隨後的《》裏被再次使用，此後在《》系列中被其他畫家大量使用。到了2000年，這已經成爲蝙蝠俠腰帶的基本通用特徵。能夠發射蝙蝠線或蝙蝠繩的抓鉤槍首次登場是在蒂姆·波頓執導的蝙蝠俠真人電影《蝙蝠俠》中，此前蝙蝠繩是需要蝙蝠俠手動拋出的。在《蝙蝠俠》及其續作電影《蝙蝠俠歸來》中，蝙蝠俠的腰帶有一個特色：通過附着在腰帶上的電機將各種裝備自動傳輸到蝙蝠俠可以拿到的地方。雖然表面看上去，蝙蝠俠的腰帶並不起眼，但這卻是蝙蝠俠打擊犯罪的重要道具之一。蝙蝠俠的腰帶主要由帶子和帶扣構成，帶子外側附着十個左右的袋子或圓筒。帶扣內部本身就隱藏着一個微型錄像機及錄音機。而帶子上的第二個帶子裏裝的就是蝙蝠俠最常用的武器之一，。十個袋子或圓筒裏都裝着蝙蝠俠必不可少的裝備或道具，這些圓筒或袋子是可以互相交換的。多年來，蝙蝠俠腰帶上的具體裝備已經發生了多次修改，以幫助蝙蝠俠針對各種犯罪。大多數的蝙蝠俠腰帶都有安全功能，以確保只有蝙蝠俠才能使用它。目前的故事中，已經確定蝙蝠俠的腰帶可以對任何接觸它的惡棍進行電擊。它也可以釋放眩暈氣體以防止被更改。腰帶上的袋子和圓筒都有鎖定功能，以禁止蝙蝠俠以外的人打開它們。蝙蝠俠腰帶上所裝備的工具非常多， 包括但不限於下面所列舉的。"
}
 2 71183 13.85200
{
  "id" : 71183,
  "contents" 

## 計算IDF值

In [ ]:
import math

def get_idf(word, df, total_docs):
    # 計算 IDF 值
    idf = math.log10(total_docs / (df + 1))
    return idf

In [ ]:
idf = get_idf( '蝙蝠俠', len(hits), total_docs)
print(idf)

5.043411939462167
